# ナイーブベイズ分類器による文書分類

In [1]:
from prettyprint import pp

## 1. 形態素解析

In [2]:
import MeCab
mecab = MeCab.Tagger('mecabrc')
text = "はじめに こんにちは。システム開発部の中村です。 機械学習についての理解を促進するため、 データから分類モデルを自動で構築する古典的な方法である"
node = mecab.parseToNode(text)
# while node:
#     if node.feature.split(',')[0] == '名詞':
#         print node.surface.lower()
#     node = node.next
    
def get_words_list(text):
    mecab = MeCab.Tagger('mecabrc')
    node = mecab.parseToNode(text)
    words_list = []
    while node:
        if node.feature.split(",")[0] == '名詞':
            words_list.append(node.surface.lower())
        node = node.next
    return words_list

def get_words_matrix(texts_list):
    """
    texts_list : ['text1', 'text2',... ]
    """
    res = []
    return [get_words_list(text) for text in texts_list]

### mecabの例

In [3]:
text1 = """
物語は、女性ファッション誌の編集者になることを夢見る女性・河野悦子が、総合出版社・景凡社の校閲部の社員として採用されるというもの。「自分のいるべき場所はここじゃない」と思いつつも、仕事にまい進する校閲者・悦子の“地味にスゴイ”活躍を描く。
物語に入っていくための丁寧な導入と、細かな“リアルさ”をぶっ飛ばす石原さとみというキャスティングの妙が、『校閲ガール』の好評ぶりにつながっているのだろう。
自動運転車の開発競争は世界中で加熱しつつある。
"""
text2 = """
オクスフォード大学からのスピンオフにより生まれた自動運転車のスタートアップであるOxboticaが、自動運転技術を搭載したルノーにてイギリスでの公道実験を開始した。イギリス公道上を自動運転車が走るのは初めてのことだ。場所はイングランドのミルトン・キーンズで、走行速度は時速5マイルだとのこと。まだ試験段階ではあるものの、自動運転車は広がりつつある
"""
res = get_words_matrix([text1,text2])
print str(res).decode('string-escape')

[['物語', '女性', 'ファッション', '誌', '編集', '者', 'こと', '女性', '河野', '悦子', '総合', '出版', '社', '景', '凡社', '校閲', '部', '社員', '採用', 'もの', '自分', '場所', 'ここ', '仕事', 'まい進', '校閲', '者', '悦子', '地味', 'スゴイ', '活躍', '物語', 'ため', '丁寧', '導入', '細か', 'リアル', 'さ', '石原', 'さとみ', 'キャスティング', '妙', '校閲', 'ガール', '好評', 'ぶり', 'の', '自動', '運転', '車', '開発', '競争', '世界中', '加熱'], ['オクスフォード', '大学', 'スピンオフ', '自動', '運転', '車', 'スタート', 'アップ', 'oxbotica', '自動', '運転', '技術', '搭載', 'ルノー', 'イギリス', '公道', '実験', '開始', 'イギリス', '公道', '上', '自動', '運転', '車', 'の', 'こと', '場所', 'イングランド', 'ミルトン・キーンズ', '走行', '速度', '時速', '5', 'マイル', 'こと', '試験', '段階', '自動', '運転', '車']]


## 2. 特徴ベクトルの作成

In [4]:
from gensim import corpora, matutils

import numpy as np


dictionary = corpora.Dictionary(res)


bows = dictionary.doc2bow(res[0]) # [(w_id1, w_id1_cnt), (w_id2, w_id2_cnt),...]
bows = [dictionary.doc2bow(x) for x in res]
# print '----- BoW -------'
#print bows
# print 
train_X = np.array([(matutils.corpus2dense([vec], num_terms=len(dictionary)).T[0]) 
                   for vec in bows])
train_y = np.array([1,0])
print '------- 特徴ベクトル --------'
print train_X


------- 特徴ベクトル --------
[[ 1.  2.  1.  1.  1.  3.  1.  1.  1.  1.  2.  1.  1.  1.  1.  1.  1.  1.
   1.  2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   2.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  4.  0.  0.  0.  1.  0.  0.
   0.  0.  0.  0.  4.  0.  3.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  1.  2.  0.  0.  0.  0.  0.  1.  1.  1.  2.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  2.  1.  1.  1.  1.  1.  1.]]


## 3. ナイーブベイズ分類器

In [5]:
# sklearnで動作確認
from sklearn.naive_bayes import MultinomialNB

### randomな例（ラベル種類数 : 3、特徴ベクトル次元数 : 40）

In [6]:
random_train_X = np.random.randint(4, size=(100,40))#cntは最大4とする
random_train_y = np.random.randint(3,size=100) # 正解ラベル数は3
random_test_X = np.random.randint(3,size=(20,40))

#### sklearn

In [7]:
clf = MultinomialNB(alpha=0.1)
clf.fit(random_train_X,random_train_y)

print clf.predict(random_test_X)
#print clf.predict_proba(random_test_X)
# print clf.predict_log_proba(random_test_X)

[0 0 0 0 0 1 2 1 1 0 1 2 2 0 1 1 1 0 2 1]


#### 自分で実装

In [8]:
def calc_cat(y): # calculate P(cat)
    label_kinds = len(np.unique(y))
    ans = np.empty(label_kinds)
    for i in range(label_kinds):
        ans[i] = len(np.argwhere(y == i)[:,0]) / float(len(y))
    return ans

def calc_each_word_bar_cat(X, y): # calculate P(word_id1 | cat)
    label_kinds = len(np.unique(y))
    ans = np.empty((label_kinds, X.shape[1]))
    for i in range(label_kinds):
        index = np.argwhere(random_train_y == i)[:,0] # index
        ans[i] = (X[index, :].sum(axis=0) + 1).astype(np.float32) / (X[index, :].sum() + X.shape[1]) # lablace smoothing
    return ans

def calc_log_prob(X, y, word_cat_prob, cat_prob, doc): # doc : 1 feature vector
        doc_word_index = np.where(doc != 0)
        each_word_prob_in_doc  = word_cat_prob[:, doc_word_index[1]] #docに含まれる単語の各P(word | cat)
        each_log_word_cat_prob = np.log(each_word_prob_in_doc).sum(axis=1)
        ans = np.log(cat_prob) + each_log_word_cat_prob
        return ans

cat_prob = calc_cat(random_train_y)
print cat_prob
print 
word_cat_prob = calc_each_word_bar_cat(random_train_X, random_train_y)
doc = np.random.randint(3,size=(1,40))
print doc
print
print calc_log_prob(random_train_X,random_train_y,word_cat_prob,cat_prob, doc)
# 

[ 0.34  0.36  0.3 ]

[[2 0 1 0 2 2 1 1 0 0 1 1 1 1 1 0 2 0 0 0 0 1 0 0 1 2 2 0 0 2 0 0 2 2 2 1 1
  1 0 0]]

[-86.01545709 -86.38800665 -87.0905958 ]


In [9]:
cat_prob = calc_cat(random_train_y)
word_cat_prob = calc_each_word_bar_cat(random_train_X, random_train_y)
res = []
docs = random_test_X
for i in range(len(docs)):
    log_prob = calc_log_prob(random_train_X,random_train_y,word_cat_prob,cat_prob, (docs[i])[np.newaxis,:])
    res.append(np.argmax(log_prob))
    
print clf.predict(random_test_X)
print np.array(res)

[0 0 0 0 0 1 2 1 1 0 1 2 2 0 1 1 1 0 2 1]
[0 0 0 0 0 1 2 1 1 0 1 2 0 0 1 1 1 0 2 1]
